In [ ]:
# My imports
from pylab import *
import numpy as np
import pyfits,glob,time
from subprocess import call
topdir='/Users/cslage/Research/LSST/code/GUI/'
forwarddir ='/Users/cslage/Research/LSST/code/galsim-newgit/GalSim/devel/brighter-fatter/forward_model_varying_i'
%cd $forwarddir
import  forward
from scipy.optimize import fmin_powell
from scipy.special import erf
global spotlist

#Sergey's imports
import ngmix
from ngmix.fitting import LMSimple, MCMCSimple
import numpy as np
import posixpath, glob, datetime, os, sys
from astropy.io import fits as pyfits
from scipy.stats import binned_statistic_2d, binned_statistic
from tqdm import tqdm,tqdm_notebook

%matplotlib inline

In [ ]:
def run_sextractor(fitsfilename):
    configfile=topdir+'sextractor/30-micron-pinholes.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    minadu=10
    outname=fitsfilename

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
 
    callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
               "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
               "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
               "-DETECT_THRESH","10","-DETECT_MINAREA","10","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","400",
               "-ANALYSIS_THRESH","2"]

    test=call(callstring)
    make_reg_from_ldac(outname+'.cat','NUMBER')



First, we're going to compare the Lage forward modeling code with ngmix MCMC on fitting GalSim generated spots.  These spots have been generated using the code build_spots_9apr18.py.  There are 100 spots with a dithered location and a nominal sigma of 1.5 pixels.

In [ ]:
thedir='/Users/cslage/Research/LSST/code/GUI/spot_sizes/'
filelist=sort(glob.glob(thedir+'bf_1.fits'))
print len(filelist)," files"


In [ ]:
tstart=time.time()
for i in range(len(filelist)): run_sextractor(filelist[i])

print time.time()-tstart

In [ ]:
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.x2=zeros([nstamps])
        self.y2=zeros([nstamps])
        self.xy=zeros([nstamps])
        self.imax=zeros([nstamps])

def BuildSpotList(fitsfilename, numspots, nx, ny, minsize, maxsize):
    global spotlist
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 0
    xcoomax = 200
    ycoomin = 0
    ycoomax = 200
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    hdr=pyfits.getheader(fitsfilename)
    img=pyfits.getdata(fitsfilename) 
    catname=fitsfilename[:-5]+'.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = img[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xoff
            spotlist.yoffset[n] = yoff

            x2sum = 0.0
            y2sum = 0.0
            xysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    x2sum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.x[i] - spotlist.xoffset[n]) * spotlist.data[i,j,n]
                    y2sum += (spotlist.y[j] - spotlist.yoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    xysum += (spotlist.x[i] - spotlist.xoffset[n]) * (spotlist.y[j] - spotlist.yoffset[n]) * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.x2[n] = x2sum / datasum
            spotlist.y2[n] = y2sum / datasum
            spotlist.xy[n] = xysum / datasum
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.x2 = spotlist.x2[0:n]
    newspotlist.y2 = spotlist.y2[0:n]
    newspotlist.xy = spotlist.xy[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

def FOM(params):
    global spotlist
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def SingleSpotList(inputspotlist, n):
    global spotlist
    stampxmin = inputspotlist.xmin
    stampxmax = inputspotlist.xmax
    nx = inputspotlist.nx
    stampymin = inputspotlist.ymin
    stampymax = inputspotlist.ymax
    ny = inputspotlist.ny
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,1)
    spotlist.xoffset = inputspotlist.xoffset[n:n+1]
    spotlist.yoffset = inputspotlist.yoffset[n:n+1]
    spotlist.x2 = inputspotlist.x2[n:n+1]
    spotlist.y2 = inputspotlist.y2[n:n+1]
    spotlist.xy = inputspotlist.xy[n:n+1]
    spotlist.data = inputspotlist.data[:,:,n:n+1]
    return spotlist

def ReadCCFile(filename, Nx, Ny):
    file = open(filename,'r')
    lines=file.readlines()
    file.close()
    lines.remove(lines[0]) # Strip the title line
    elec = zeros([Nx,Ny])
    count = 0
    for line in lines:
        items = line.split()
        i = int(items[0]) - 1
        j = int(items[1]) - 1        
        elec[i,j] = int(items[2])

    return elec


In [ ]:
# We build the list of spots with 9x9 postage stamps
spotlist = BuildSpotList(filelist[0], 100, 9, 9, 1.3, 1.7)
param0 = [1.0, 1.0]
args = ()#(spotlist)
Result = fmin_powell(FOM, param0, args)
print spotlist.nstamps
print Result


In [ ]:
# Let's look at a random spot
print spotlist.nstamps
imshow(spotlist.data[:,:,33],interpolation='Nearest')

As expected, the forward modeling code returns exactly 1.5 pixels for the best fit sigma of all 100 spots
Now let's try using NGMIX on the same spots.  I don't see how to use NGMIX to fit a whole set of spots, so we'll compare the results on one spot at a time. Note that I had to increase the guess for 2*sigma^2 in get_sigma to get it to return reasonable values.

In [ ]:
# Imported from Sergey's NGMIX notebook
# Added a sigma_guess to speed convergence
def srandu(num=None):
#  Generate random numbers in the symmetric distribution [-1,1]
    return 2*(np.random.random(num)-0.5)

def get_sigma(img, weight_map=None, xc=4.0, yc=4.0, sigma_guess=0.4, nwalkers=80, burnin=400, nstep=400):
    guess=np.zeros((nwalkers, 6))

    if weight_map is None:
        weight_map = 1./np.sqrt(np.fabs(img))
        
    obs = ngmix.observation.Observation(img, weight=weight_map)
    mc = MCMCSimple(obs, 'gauss', nwalkers=nwalkers, nsub=4)
    guess[:,0] = xc + 0.1*srandu(nwalkers)
    guess[:,1] = yc + 0.1*srandu(nwalkers)
    guess[:,2] = 0. + 0.1*srandu(nwalkers)
    guess[:,3] = 0. + 0.1*srandu(nwalkers)
    guess[:,4] = 2.0 * sigma_guess * sigma_guess * (1.0 + 0.1*srandu(nwalkers))
    guess[:,5] = np.sum(img) * (1.0 + 0.1*srandu(nwalkers))

    pos = mc.run_mcmc(guess, burnin)
    pos = mc.run_mcmc(pos,   nstep)

    mc.calc_result()

    result = mc.get_result()

    if result['flags'] == 0:
        #print result['pars']
        return result['pars'][0], result['pars'][1], np.sqrt(result['pars'][4]/2.0)
    else:
        return 0.0, 0.0, 0.0


In [ ]:
# NGMIX is very slow, so we'll just do 25 spots
fullspotlist = BuildSpotList(filelist[0], 100, 9, 9, 1.3, 1.7)

ngmix_sigma = []
forward_sigma = []
nspot = 20
numspots = 0
while numspots < 25:
    spotlist = SingleSpotList(fullspotlist, nspot)
    img = spotlist.data[:,:,0]
    # First, run the forward modeling
    param0 = [1.0, 1.0]
    args = ()#(spotlist)
    Result = fmin_powell(FOM, param0, args)

    # Next run NGMIX with MCMC
    xc,yc = 4.5, 4.5
    yc,xc,ng_sigma = get_sigma(img, xc=yc, yc=xc, sigma_guess = 1.5, nstep=400)
    if (not isnan(ng_sigma)) and ng_sigma > 1.2 and ng_sigma < 1.8:
        ngmix_sigma.append(ng_sigma)
        for_sigma = (Result[0] + Result[1])/2.0
        forward_sigma.append(for_sigma)
        print for_sigma, ng_sigma
        numspots += 1
        nspot += 1




In [ ]:
print array(ngmix_sigma).mean(), array(ngmix_sigma).std(), array(forward_sigma).mean(), array(forward_sigma).std(), 

In [ ]:
# Now let's plot the results
figure(figsize = (16,8))
title("Forward Model / NGMIX comparison on GalSim spots")
scatter(forward_sigma, ngmix_sigma, s=5.0)
xlabel("Forward model result")
ylabel("NGMIX Result")
#xlim(1.4995,1.5005)
#ylim(1.4995,1.5005)
text(1.5001,1.5006,"Forward Model Sigma = %.5f"%array(forward_sigma).mean(),fontsize = 12)
text(1.5001,1.5003,"NGMIX Sigma = %.5f"%array(ngmix_sigma).mean(),fontsize = 12)
savefig(thedir+"Forward_NGMIX_Comparison_10APR18.png")



NGMIX is much slower and has more scatter, but basically the two codes agree that the spot sizes are 1.5 microns.
So where is the discrepancy? Next I'll try to understand how Sergey converted the *Pts.dat files into images.  I have been using the *CC.dat files, which only have the number of electrons in each pixel.

In [ ]:
base = '/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Fe55/data/skrun1/Pixel'

x0s,y0s,x1s,y1s = [],[],[],[]
sigmas = []

noise = 10.0

for iter in tqdm_notebook(xrange(1)): # Process first 100 spots from given directory
    filename = '%s_%d_Pts.dat' % (base, iter)
    if posixpath.exists(filename):

        id,step,ph,x,y,z = np.loadtxt(filename, skiprows=1, unpack=True)
        
        id0 = id[ph==0]
        x0,x1 = x[ph==0], x[ph!=0] # Initial coordinates of electron
        y0,y1 = y[ph==0], y[ph!=0] # Final coordinates of electron

        gmag0, xe, ye, binnumbers = binned_statistic_2d(x0, y0, x1-x0, bins=[6,6], statistic='count', range=[[0,60],[0,60]])
        gmag1, xe, ye, binnumbers = binned_statistic_2d(x1, y1, x1-x0, bins=[6,6], statistic='count', range=[[0,60],[0,60]])
        img0 = gmag0.T # Pixelization of initial locations
        img1 = gmag1.T # Pixelization of final locations
        # Compare Sergey's pixelization to my CC file
        print img1
        ccfilename = filename = '%s_%d_CC.dat' % (base, iter)
        ccfile = ReadCCFile(filename, 9, 9)
        print ccfile[0:6,0:6].T
        figure()
        subplot(1,2,1)
        title("Pixelized Image - No Noise")
        imshow(img1, interpolation='Nearest')

        # Rough centroid estimation for Gaussian fitting
        xx,yy = np.meshgrid(np.arange(6), np.arange(6))
        xc,yc = np.sum(xx*img1)/np.sum(img1), np.sum(yy*img1)/np.sum(img1)
        
        # Add artificial read-out noise
        img = img1 + np.random.normal(scale=noise, size=img1.shape)
        subplot(1,2,2)
        title("Pixelized Image - With Noise")
        imshow(img, interpolation='Nearest')

        weight_map = np.ones_like(img)*1./noise**2

        # Get peak position using MCMC
        yc,xc,sigma = get_sigma(img, weight_map=weight_map, sigma_guess = 0.24, xc=yc, yc=xc, nstep=400)
        
        # If the fit properly converged, store this spot parameters
        if xc > 0 and xc < 6 and yc > 0 and yc < 6 and sigma > 0 and sigma < 10:
            sigmas.append(sigma)
            x0s.append(np.mean(x0))
            y0s.append(np.mean(y0))
            x1s.append(xc)
            y1s.append(yc)
            print sigma



Sergey's pixelization agrees exactly with my CC file, with an offset that won't matter.  So the problem lies elsewhere.  Notice that the sigma extracted for this one spot is much smaller than the sigma extracted from a group of spots.  What if I run my forward modeling on one of these spots?  It will probably come out much smaller.

In [ ]:
spotlist = BuildSpotList(filelist[0], 1, 5, 5, 1.3, 1.7)
spotlist.data[:,:,0] = img1[1:6,1:6]

param0 = [1.0, 1.0]
args = ()#(spotlist)
Result = fmin_powell(FOM, param0, args)
print Result, (Result[0] + Result[1]) / 2.0


Now let's try it on an actual run.

In [ ]:
base = '/Users/cslage/Research/LSST/code/poisson/Poisson_CCD_Hole20_Fe55/data/llnl_fe55_16/cc_files/'

x0s,y0s,x1s,y1s = [],[],[],[]
sigmas = []

noise = 10.0

stacked_image = zeros([6,6])

for iter in range(128): # Process first 64 spots from given directory
    filename = base+'run_%d/Pixel_0_Pts.dat' % iter
    if posixpath.exists(filename):
        try:
            id,step,ph,x,y,z = np.loadtxt(filename, skiprows=1, unpack=True)
        
            id0 = id[ph==0]
            x0,x1 = x[ph==0], x[ph!=0] # Initial coordinates of electron
            y0,y1 = y[ph==0], y[ph!=0] # Final coordinates of electron

            gmag0, xe, ye, binnumbers = binned_statistic_2d(x0, y0, x1-x0, bins=[6,6], statistic='count', range=[[0,60],[0,60]])
            gmag1, xe, ye, binnumbers = binned_statistic_2d(x1, y1, x1-x0, bins=[6,6], statistic='count', range=[[0,60],[0,60]])
            img0 = gmag0.T # Pixelization of initial locations
            img1 = gmag1.T # Pixelization of final locations
            stacked_image = stacked_image + img1
            # Rough centroid estimation for Gaussian fitting
            xx,yy = np.meshgrid(np.arange(6), np.arange(6))
            xc,yc = np.sum(xx*img1)/np.sum(img1), np.sum(yy*img1)/np.sum(img1)
        
        # Add artificial read-out noise
            img = img1 + np.random.normal(scale=noise, size=img1.shape)
            subplot(2,2,3)
            imshow(img, interpolation='Nearest')

            weight_map = np.ones_like(img)*1./noise**2

            # Get peak position using MCMC
            yc,xc,sigma = get_sigma(img, weight_map=weight_map, sigma_guess = 0.24, xc=yc, yc=xc, nstep=400)
        
            # If the fit properly converged, store this spot parameters
            if xc > 0 and xc < 6 and yc > 0 and yc < 6 and sigma > 0 and sigma < 10:
                sigmas.append(sigma)
                x0s.append(np.mean(x0))
                y0s.append(np.mean(y0))
                x1s.append(xc)
                y1s.append(yc)
                print sigma
        except:
            continue


        
sigmas,x0s,y0s,x1s,y1s = [np.array(_) for _ in sigmas,x0s,y0s,x1s,y1s]

In [ ]:
subplot(211)
hist(sigmas, bins=10);
xlabel('Sigma')
xlim(0.0,0.5)
text(0.01,20.0,"Mean Sigma = %.3f"%sigmas.mean())
text(0.01,15.0,"Sigma Sigma = %.3f"%sigmas.std())
savefig(thedir+"Spot_by_Spot_Sigmas_Run16_10Apr18.png")

In [ ]:
imshow(stacked_image, interpolation='Nearest')
savefig(thedir+"Stacked_Map_Run16_10Apr18.png")

In [ ]:
stacked_image = stacked_image + np.random.normal(scale=noise, size=stacked_image.shape)
weight_map = np.ones_like(stacked_image)*1./noise**2

yc,xc,sigma = get_sigma(stacked_image, weight_map=weight_map, xc=3.0, yc=3.0, nstep=40)

In [ ]:
print "Run 14, Electrons and Holes Mutual Interactions", xc, yc, sigma

In [ ]:
print "Run 15, Only Electron Mutual interactions", xc, yc, sigma

In [ ]:
print "Run 16, No Mutual Interactions", xc, yc, sigma

In [ ]:
# Check on Gaussian fit as compared to Ivan's paper
Nx = Ny = 5
yindex = Ny / 2
stampxmin = -(int(Nx/2)+0.5)
stampxmax = -stampxmin
stampymin = -(int(Ny/2)+0.5)
stampymax = -stampymin
spotlist = Array2dSet(stampxmin,stampxmax,Nx,stampymin,stampymax,Ny,1)
sigma = 0.44
gauss = zeros([Nx, Ny])
NormFactor = 1620.0 / Area(spotlist.x[0]-0.5, spotlist.x[-1]+0.5, spotlist.y[0]-0.5, spotlist.y[-1]+0.5, sigma, sigma, 1)
for i in range(Nx):
    for j in range(Ny):
        gauss[i, j] = Area(spotlist.x[i]-0.5, spotlist.x[i]+0.5, spotlist.y[j]-0.5, spotlist.y[j]+0.5, sigma, sigma, NormFactor)
figure()
subplot(1,1,1,aspect = 1)
suptitle("Fe55 spot check Sigma = %.3f pixels"%sigma)

for i in range(Nx):
    plot([spotlist.x[i]-0.5, spotlist.x[i]-0.5], [spotlist.y[0]-0.5, spotlist.y[Ny-1]+0.5], color = 'black', ls = '--')
    plot([spotlist.x[i]+0.5, spotlist.x[i]+0.5], [spotlist.y[0]-0.5, spotlist.y[Ny-1]+0.5], color = 'black', ls = '--')
for j in range(Ny):
    plot([spotlist.x[0]-0.5, spotlist.x[Nx-1]+0.5], [spotlist.y[j]-0.5, spotlist.y[j]-0.5], color = 'black', ls = '--')
    plot([spotlist.x[0]-0.5, spotlist.x[Nx-1]+0.5], [spotlist.y[j]+0.5, spotlist.y[j]+0.5], color = 'black', ls = '--')


for i in range(Nx):
    for j in range(Ny):
        text(spotlist.x[i] - 0.35, spotlist.y[j]-0.10, "%.1f"%gauss[i,j], color = 'red') 

text(spotlist.x[Nx-1] + 1.2, spotlist.y[yindex] - 0.25, "Gaussian Fit", color = 'red') 
text(spotlist.x[Nx-1] + 1.2, spotlist.y[yindex] - 0.55, "$\sigma$=%.3f pixels"%sigma,color = 'red') 


savefig(thedir+"Gaussian_Model_Check_11Apr18.pdf")
print spotlist.data.sum()
